In [2]:
pip install tensorflow opencv-python


Note: you may need to restart the kernel to use updated packages.


In [1]:
import os
import cv2
import numpy as np
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense
from sklearn.model_selection import train_test_split

In [2]:
data = []
labels = []


In [3]:
for category in ['with_mask', 'without_mask']:
    path = os.path.join("D:\project\data", category)
    label = 0 if category == 'with_mask' else 1


<>:2: SyntaxWarning: invalid escape sequence '\p'
<>:2: SyntaxWarning: invalid escape sequence '\p'
C:\Users\sownd\AppData\Local\Temp\ipykernel_16488\4110348395.py:2: SyntaxWarning: invalid escape sequence '\p'
  path = os.path.join("D:\project\data", category)


In [4]:
    for img in os.listdir(path):
        img_path = os.path.join(path, img)
        img_arr = cv2.imread(img_path)
        if img_arr is not None:
            resized = cv2.resize(img_arr, (100, 100))
            data.append(resized)
            labels.append(label)


In [5]:
X = np.array(data)/255.0
y = np.array(labels)


In [6]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)


In [7]:
model = Sequential([
    Conv2D(32, (3,3), activation='relu', input_shape=(100, 100, 3)),
    MaxPooling2D(2,2),
    Conv2D(64, (3,3), activation='relu'),  # <--- NEW LAYER
    MaxPooling2D(2,2),
    Flatten(),
    Dense(64, activation='relu'),
    Dense(1, activation='sigmoid')
])

C:\Users\sownd\anaconda3\Lib\site-packages\keras\src\layers\convolutional\base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [8]:
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])


In [ ]:
model.fit(X_train, y_train, epochs=20, validation_split=0.1)


Epoch 1/20
87/87 ━━━━━━━━━━━━━━━━━━━━ 14s 130ms/step - accuracy: 0.9569 - loss: 0.0407 - val_accuracy: 1.0000 - val_loss: 1.6584e-18
Epoch 2/20
87/87 ━━━━━━━━━━━━━━━━━━━━ 11s 126ms/step - accuracy: 1.0000 - loss: 1.4852e-13 - val_accuracy: 1.0000 - val_loss: 1.6384e-18
Epoch 3/20
87/87 ━━━━━━━━━━━━━━━━━━━━ 20s 124ms/step - accuracy: 1.0000 - loss: 2.3183e-12 - val_accuracy: 1.0000 - val_loss: 1.6384e-18
Epoch 4/20
87/87 ━━━━━━━━━━━━━━━━━━━━ 20s 123ms/step - accuracy: 1.0000 - loss: 1.0793e-12 - val_accuracy: 1.0000 - val_loss: 1.6384e-18
Epoch 5/20
87/87 ━━━━━━━━━━━━━━━━━━━━ 11s 123ms/step - accuracy: 1.0000 - loss: 1.7177e-12 - val_accuracy: 1.0000 - val_loss: 1.6384e-18
Epoch 6/20
87/87 ━━━━━━━━━━━━━━━━━━━━ 20s 123ms/step - accuracy: 1.0000 - loss: 2.3183e-12 - val_accuracy: 1.0000 - val_loss: 1.6384e-18
Epoch 7/20
87/87 ━━━━━━━━━━━━━━━━━━━━ 21s 123ms/step - accuracy: 1.0000 - loss: 7.7475e-13 - val_accuracy: 1.0000 - val_loss: 1.6384e-18
Epoch 8/20
87/87 ━━━━━━━━━━━━━━━━━━━━ 11s 124

In [ ]:
model.save('mask_model.h5')


In [ ]:
from tensorflow.keras.models import load_model

model = load_model('mask_model.h5')


In [ ]:
import cv2
import numpy as np

# Change this to your test image path
img_path = "D:\project\data\without_mask\without_mask_304.jpg"

# Load and prepare the image
img = cv2.imread(img_path)
resized = cv2.resize(img, (100, 100))
normalized = resized / 255.0
reshaped = np.reshape(normalized, (1, 100, 100, 3))  # Add batch dimension


In [ ]:
prediction = model.predict(reshaped)

if prediction[0][0] < 0.5:
    print("😷 Mask Detected")
else:
    print("❌ No Mask Detected")


In [ ]:
model.evaluate(X_test, y_test)


In [ ]:
import matplotlib.pyplot as plt

plt.imshow(cv2.cvtColor(resized, cv2.COLOR_BGR2RGB))
plt.title("Prediction: Mask" if prediction[0][0] < 0.5 else "Prediction: No Mask")
plt.axis('off')
plt.show()


In [ ]:
prediction = model.predict(reshaped)
print("Prediction confidence:", prediction[0][0])
